# The ideal classical gas subject to gravity

### Lead: Bart Hoogenboom, University College London

### Author: Luke Davis, University College London

### Running instructions

The notebook runs from top to bottom and you should ``Run`` each input cell one after the other. To run the interactive plot please press the ``Run Interact`` button after executing the code cell containing ``@interact_manual``.

## Introduction:

Consider an ideal classical gas consisting of $n$ point-like particles of mass $m$ at a temperature $T$. Imagine such a gas at the surface of the earth where the gravitational field strength $g$ is $\approx 9.81$ m/s$^2$, and the particles are located at certain heights $h \geq 0$.

The energy of one particle, say $i$, of this gas is given as

$$E_i(h) = \frac{3}{2} k_B T + mgh, $$

where $k_B$ is Boltzmann's constant. The first term in the energy is the thermal energy and the second term is the gravitational potential energy. Dividing this expression by $k_B T$ (making it dimensionless) leads to

$$\frac{E_i(h)}{k_B T} = \frac{3}{2} + \frac{mgh}{k_B T}, $$

in which the behaviour of the particle, at a given height $h$, is dictated by the ratio of its weight $mg$ to the thermal background $k_B T$. The $n$ particles are distributed above the surface according to

$$ n_v (h) = n_0 \exp \left( - \frac{mg}{k_B T} h \right), $$

where $n_v$ and $n_0$ are, respectively, the number of particles at a height $h$ and the number of particles coincident with the earth's surface.

If we restrict our considerations to a system with maximum height $H$, the total number of particles (by definition) is given as

$$ n = \int_0^H n_v(h) dh = n_0 \int_0^H \exp \left( - \frac{mgh}{k_B T} \right) dh. $$

In the code below you will vary two quantities (1) the overall number of gas particles $n$ and (2) the gravitational potential energy at the maximum height $H$ ($mgh$). You will investigate how the overall number of gas particles affects the appearance of layering in the system.

## Pre-amble code

In [654]:
%%capture
#!conda install -y -q ffmpeg
import sys # python system library
import numpy as np # numerical python library
import math # python math library
import matplotlib as mpl # matplotlib shorthand
import matplotlib.pyplot as plt # matplotlib shorthand
from mpl_toolkits.mplot3d import Axes3D # 3D plots
import matplotlib.animation as animation # animation (video)
import matplotlib.gridspec as gridspec # fancy subplots
from IPython.display import HTML # shows animation in jupyter
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [655]:
matplotlib inline

In [656]:
# Plot parameters
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['figure.dpi'] = 150
plt.rcParams["font.family"] = "Serif"
params = {'mathtext.default': 'regular' }  

## Code

In [657]:
H=100.0 # Maximum height limit 
dh=0.5  # Height width for the particle plot

In [658]:
def nv(x,n0,mgkT): # number density function as defined in the introduction 
    # x is an array of distances perpendicular to ground
    # n0 = number particles coincident with ground
    # mgkT = mg/kT
    return (n0*np.exp(-(mgkT*x))) # return an array of the outputs from the number density equation

In [659]:
def particles(x,n0,mgkT): # function to generate gas particles
    points = []
    xcnt = 0
    nums = nv(x,n0,mgkT) # outputs from the number density equation
    
    for i in nums: # loop over the number of particles at a particular height
        for n in range(1,math.ceil(i)): # this loop collects the heights of the particles
            points.append(x[xcnt])
        xcnt += 1
    return points # return the heights of the points

def histogram(bins,data): # function to histogram generated heights of the points
    histo = []
    for x in bins:
        cnt = 0
        for height in data:
            if height == x:
                cnt += 1 # add to histogram
        histo.append(cnt)
    return histo # return array of the counts

In [660]:
@interact_manual(n=widgets.IntSlider(min=50,max=5000,step=50),
                 mgh=widgets.FloatSlider(min=5,max=100,step=5)) # variables to change
def IdealGasGravity(n,mgh): # main function for the interactive plot
    bins = np.arange(0,H,dh) # the discrete heights

    mgkT = mgh/H # the mg/kT quantity

    n0 = (int(n)/(np.sum(np.exp(-(mgkT*bins))))) # finding n0 from the specified n

    fig, axs = plt.subplots(1,2)
    
    particleHeights=particles(bins,n0,mgkT) # generate particle heights
    
    nparticles = len(particleHeights) # actual number of collect points
    
    ### Plotting ###
    
    axs[0].set(xlabel='Horizontal distance',ylabel='Height',title='Ideal gas subject to gravity')
    axs[0].set_ylim(0.0,H)
    axs[0].set_xlim(0.0,10)
    axs[0].tick_params(which='both',direction='out',right=True,top=False)
    axs[0].set_xticklabels('')
    axs[0].set_yticklabels(['0','','','','','Max'])
    axs[0].scatter(np.random.uniform(low=0.0,high=10.0,size=len(particleHeights)),particleHeights,color='orange'
                   ,edgecolor='black',alpha=0.7,label='$\it{n}=$'+str(nparticles))
    axs[0].set_facecolor('xkcd:sky blue')
    axs[0].legend(loc='best') #plot legend
    
    histbins = np.arange(0,H,5)
    
    axs[1].bar(histbins,histogram(histbins,particleHeights),
               edgecolor='black',width=5,color='orange',alpha=0.7,label='Observed (discrete)')
    
    axs[1].set(xlabel='Height',ylabel='Particle count',title='')
    axs[1].set_xlim(0.0,H)
    axs[1].plot(bins,nv(bins,n0,mgkT),color='black',label="Theory (continuous)")
    axs[1].legend(loc='best') #plot legend
    
   
    
    fig.tight_layout(pad=1.0)

interactive(children=(IntSlider(value=50, description='n', max=5000, min=50, step=50), FloatSlider(value=5.0, …

Note that, due to the discreteness of converting the continuous distribution $n_v$ to individual points (left figure) the number of shown points in the plot is $< n$ (as determined on the slider). For large $n$, e.g., $n > 1000$, this becomes less significant. The effect also becomes less significant at higher gravitational potential energy, since a large portion of $n_v$ is greater than 1 (anything less than 1 is counted as nothing).

## Conclusions

In summary, the interactive plots reveal that for low numbers of gas particles, the layering/gradient effects that arise due to the gravitational pulling of the gas towards the earth are barely noticeable. However, upon increasing the particle numbers the effects become easier to see, this is an important lesson in statistical physics: patterns and laws sometimes require large numbers of particles to become noticeable.